In [10]:
import pandas as pd
import numpy as np
import sys 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import itertools
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import Parallel, delayed, dump, load
# sys.path.insert(0, '../DevCode')


pd.set_option('display.expand_frame_repr', False)
pd.options.display.max_rows = 500
sys.path.append('../src')
import pickle

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim


from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR


<h2> Read in the Scaled Data and Assign it </h2>

In [12]:
X_test = np.load('../src/data/scaled_test_features.npz')['a']
y_test = np.load('../src/data/scaled_test_labels.npz')['a']

In [13]:
X_train = np.load('../src/data/scaled_train_features.npz')['a']
y_train = np.load('../src/data/scaled_train_labels.npz')['a']

In [17]:
X_val = np.load('../src/data/scaled_valid_features.npz')['a']
y_val = np.load('../src/data/scaled_valid_labels.npz')['a']

<h2> To help increase accuracy of the model, I reversed the sequences so that <br> the focus lies on the initial portions of the URL String and those important items become the end of the sequence</h2>

In [18]:
X_train = X_train[:, ::-1].copy()
X_val = X_val[:, ::-1].copy()
X_test = X_test[:, ::-1].copy()

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

<h2> For the Model Architcture I desiced to use a Stacked LSTM. <br> 
The core LSTM feeds into a general Multi Layer Perceprtion and then utilizes <br>
the ReLU activation function to help theoretically increase pattern recognition</h2>

In [61]:
class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_rate):
        super(StackedLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        # self.fc = nn.Linear(hidden_size, output_size)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

<h2> For the Model Architcture I desiced to use a Bi Directional LSTM. <br> 
</h2>

In [20]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # We multiply by 2 for bidirectional

        self.fc2 = nn.Linear(hidden_dim * 2, hidden_dim)

        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Assuming x is already in the correct shape: [batch_size, seq_len, input_dim]
        lstm_out, (hidden, cell) = self.lstm(x)
        # Concatenate the final forward and backward hidden states
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        out = self.fc2(hidden)
        out = self.relu(out)
        out = self.fc3(out)
        return out

<h2> Assign and feature and label datasets and reshape them for LSTM Architecture</h2>

In [21]:
features = X_train
labels = y_train

features = features.reshape(-1, 47,1)
labels = labels.reshape(-1)

<h2>Load the dataset into our Data Loader clas </h2>

In [22]:


# Convert to tensors
features_tensor = torch.tensor(features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

dataset = TensorDataset(features_tensor, labels_tensor)
data_loader = DataLoader(dataset, batch_size=1024, shuffle=True)

<h2> Assign our HyperParameters </h2>
<h3>I decided to use a number of neurons equivelant to <br> 
twice the amount of samples in a sequence<br>
<h3> I then selected a Step Scheduled Learning rate which would <br>
cut the learning rate in half every 25 epochs

In [23]:
input_dim = 1
hidden_dim = 96
output_dim = 4  # Assuming binary classification
num_layers = 2


learing_rate = 0.01


model = BiLSTMClassifier(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=learing_rate)

scheduler = StepLR(optimizer, step_size=25, gamma=0.5)

In [25]:
model.load_state_dict(torch.load('../src/data/model2.pt'))

<All keys matched successfully>

<h2> Perform our training

In [20]:
num_epochs = 300
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Forward pass

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    scheduler.step()



        # inputs = inputs.cpu()
        # labels = labels.cpu()
        

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/300], Loss: 0.4716
Epoch [2/300], Loss: 0.2897
Epoch [3/300], Loss: 0.2014
Epoch [4/300], Loss: 0.2678
Epoch [5/300], Loss: 0.2281
Epoch [6/300], Loss: 0.2933
Epoch [7/300], Loss: 0.1862
Epoch [8/300], Loss: 0.2039
Epoch [9/300], Loss: 0.1755
Epoch [10/300], Loss: 0.2792
Epoch [11/300], Loss: 0.1681
Epoch [12/300], Loss: 0.1253
Epoch [13/300], Loss: 0.1533
Epoch [14/300], Loss: 0.1534
Epoch [15/300], Loss: 0.1701
Epoch [16/300], Loss: 0.0980
Epoch [17/300], Loss: 0.1888
Epoch [18/300], Loss: 0.1163
Epoch [19/300], Loss: 0.0971
Epoch [20/300], Loss: 0.1406
Epoch [21/300], Loss: 0.0820
Epoch [22/300], Loss: 0.0855
Epoch [23/300], Loss: 0.1572
Epoch [24/300], Loss: 0.1258
Epoch [25/300], Loss: 0.2171
Epoch [26/300], Loss: 0.1454
Epoch [27/300], Loss: 0.0602
Epoch [28/300], Loss: 0.1244
Epoch [29/300], Loss: 0.0952
Epoch [30/300], Loss: 0.1116
Epoch [31/300], Loss: 0.0893
Epoch [32/300], Loss: 0.1099
Epoch [33/300], Loss: 0.0984
Epoch [34/300], Loss: 0.1318
Epoch [35/300], Loss: 0

<h2> Save our Models and Optimizer for later use

In [21]:
torch.save(model.state_dict(), '../src/data/model2.pt')
torch.save(optimizer.state_dict(), '../src/data/optimizer2.pt')

<h2> Assign our Testing or Validation Data

In [29]:
val_bool = True

if val_bool:
    valid_features = X_val
    valid_labels = y_val
else:
    valid_features = X_test
    valid_labels = y_test


valid_features = valid_features.reshape(-1, 47,1)
valid_labels = valid_labels.reshape(-1)

valid_features_tensor = torch.tensor(valid_features, dtype=torch.float32)
valid_labels_tensor = torch.tensor(valid_labels, dtype=torch.long)


valid_dataset = TensorDataset(valid_features_tensor, valid_labels_tensor)
valid_data_loader = DataLoader(dataset, batch_size=1024, shuffle=True)

<h2> Create a function to run the testing or validation

In [30]:
def validate(model, val_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to track the gradients
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    return val_loss, accuracy


<h2> Run and analyze the testing or validation data and retrieve accuracy

In [31]:
# Example usage
val_loss, val_accuracy = validate(model, valid_data_loader, criterion)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')


Validation Loss: 0.0243, Validation Accuracy: 99.25%
